# Toronto Clustering Assignment
## Applied Data Science Capstone
### Week 3

**Name**: Robert Barrimond

**Date**: May 8, 2021

**REVIEWER PLEASE NOTE**
I do _not_ comment my code with Markdown. As an SRE, I do as application developers should do: document code _in the code_ and everywhere possible by the code itself. Having said that, SREs are also called to be data scientists as well. So, I use Markdown to "tell the story" as that first overview course taught me so many months ago. I hope this assignment was easy to follow and grade!

---

### Scrape Raw Data from Wikipedia
The first step is to get the raw data and since I live in the Pandas world in my day job, I'll stick to the knitting here so `pd.read_html` is my first step. Then since the table has no structure we need to preserve, we can flatten it to a Series and extract the columns one by one resulting in the final Dataframe object. Of course, there is some minor manual editing of anamolous data along the way. 

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup 

In [2]:
# Scrape Wikipedia and get the table into a data frame.
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
beautiful_soup = BeautifulSoup(html_data)
fsa_raw_df = pd.read_html(beautiful_soup.find_all('table')[0].prettify(), flavor='bs4')[0]
fsa_raw_df.head()

,0,1,2,3,4,5,6,7,8
0,M1A Not assigned,M2A Not assigned,M3A North York ( Parkwoods ),M4A North York ( Victoria Village ),M5A Downtown Toronto ( Regent Park / Harb...,M6A North York ( Lawrence Manor / Lawrenc...,M7A Queen's Park (Ontario Provincial Governm...,M8A Not assigned,M9A Etobicoke ( Islington Avenue )
1,M1B Scarborough ( Malvern / Rouge ),M2B Not assigned,M3B North York ( Don Mills ) North,M4B East York ( Parkview Hill / Woodbine ...,"M5B Downtown Toronto ( Garden District , ...",M6B North York (Glencairn),M7B Not assigned,M8B Not assigned,M9B Etobicoke ( West Deane Park / Princes...
2,M1C Scarborough ( Rouge Hill / Port Union...,M2C Not assigned,M3C North York ( Don Mills ) South ( Fl...,M4C East York ( Woodbine Heights ),M5C Downtown Toronto ( St. James Town ),M6C York ( Humewood-Cedarvale ),M7C Not assigned,M8C Not assigned,M9C Etobicoke (Eringate / Bloordale Gardens ...
3,M1E Scarborough ( Guildwood / Morningside...,M2E Not assigned,M3E Not assigned,M4E East Toronto ( The Beaches ),M5E Downtown Toronto (Berczy Park),M6E York ( Caledonia-Fairbanks ),M7E Not assigned,M8E Not assigned,M9E Not assigned
4,M1G Scarborough ( Woburn ),M2G Not assigned,M3G Not assigned,M4G East York ( Leaside ),M5G Downtown Toronto (Central Bay Street ),M6G Downtown Toronto (Christie),M7G Not assigned,M8G Not assigned,M9G Not assigned


In [3]:
# Flatten the data and produce the Series object
fsa_list = pd.Series(fsa_raw_df.values.flatten())
fsa_list.drop(fsa_list.loc[fsa_list.str.match(r'^\s*M\w{2}\s+Not assigned')].index, inplace=True) # Drop all unassigned FSAs
fsa_list.reset_index(drop=True, inplace=True)
fsa_list[0:10]

0                     M3A  North York  (  Parkwoods  )
1              M4A  North York  (  Victoria Village  )
2    M5A  Downtown Toronto  (  Regent Park  /  Harb...
3    M6A  North York  (  Lawrence Manor  /  Lawrenc...
4    M7A  Queen's Park  (Ontario Provincial Governm...
5               M9A  Etobicoke  (  Islington Avenue  )
6            M1B  Scarborough  (  Malvern  /  Rouge  )
7              M3B  North York  (  Don Mills  )  North
8    M4B  East York  (  Parkview Hill  /  Woodbine ...
9    M5B  Downtown Toronto  (  Garden District  ,  ...
dtype: object

In [4]:
# Begin building the final Dataframe. First extract the postal codes and boroughs.
fsa_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
fsa_df['PostalCode'] = fsa_list.str.extract(r'^(M\w{2})\s{2}', expand=False)
fsa_df['Borough'] = fsa_list.str.extract(r'^\s*M\w{2}\s+(.+)\s\(', expand=False).str.strip()
fsa_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,NaN
1,M4A,North York,NaN
2,M5A,Downtown Toronto,NaN
3,M6A,North York,NaN
4,M7A,Queen's Park,NaN
5,M9A,Etobicoke,NaN
6,M1B,Scarborough,NaN
7,M3B,North York,NaN
8,M4B,East York,NaN
9,M5B,Downtown Toronto,NaN


In [5]:
# Some "manual" cleanup on the boroughs
fsa_df.loc[[13, 40], 'Borough'] = 'North York'
fsa_df.loc[92, 'Borough'] = 'Downtown Toronto Stn A'
fsa_df.loc[100, 'Borough'] = 'East Toronto Business'
fsa_df.loc[35, 'Borough'] =  'East York/East Toronto'
fsa_df.loc[76, 'Borough'] = 'Mississauga'
fsa_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,NaN
1,M4A,North York,NaN
2,M5A,Downtown Toronto,NaN
3,M6A,North York,NaN
4,M7A,Queen's Park,NaN
5,M9A,Etobicoke,NaN
6,M1B,Scarborough,NaN
7,M3B,North York,NaN
8,M4B,East York,NaN
9,M5B,Downtown Toronto,NaN


In [6]:
# Extract the neighborhoods
fsa_df['Neighborhood'] = fsa_list.str.extract(r'^.+\((.*)\)', expand=False)
fsa_df['Neighborhood'] = fsa_df['Neighborhood'].str.split(r'[\,/]')
fsa_df.loc[13,'Neighborhood'].append('Don Mills')
fsa_df.loc[40,'Neighborhood'].append('Downsview')
fsa_df['Neighborhood'] = fsa_df['Neighborhood'].apply(lambda x: list(map(str.strip, x)))
fsa_df['Neighborhood'] = fsa_df['Neighborhood'].apply(lambda x: ", ".join(x))
fsa_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Personally, I would have left the Neighborhood column as a list rather than a string of comma separated values. A list is more powerful and can be display just as prettily as a flat string but assignments are assignments...🤷🏽‍♂️

---

### Add Geocoding
Because of the caveats with Geocoder retrieving geocoding, grab the CSV and cover myself. Then give try Geocoder per the assignment and see if it works out. It didn't. So then try GeoPy and Nominatim which ended with mixed results.

In [7]:
fsa_geo_coord_df = pd.read_csv('Geospatial_Coordinates.csv')
fsa_geo_coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
fsa_df = fsa_df.join(fsa_geo_coord_df.set_index('Postal Code'), on='PostalCode')
fsa_df.to_pickle('fsa_df.pkl')
fsa_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [9]:
# Null element check
fsa_df.loc[fsa_df.isna()['Latitude'] | fsa_df.isna()['Longitude']]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [10]:
import geocoder
# initialize your variable to None
lat_lng_coords = None
iterations = 0

# loop until you get the coordinates
while( (lat_lng_coords is None) & (iterations < 100) ):
    g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
    lat_lng_coords = g.latlng
    iterations+=1
    # print("Iteration: ", iterations)

if(lat_lng_coords == None):
    print("No data returned")
else:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

No data returned


In [11]:
import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='robs_ba_explorer')
for fsa, borough in zip(fsa_df['PostalCode'], fsa_df['Borough']):
        address = '{}, Ontario'.format(fsa)
        location = geolocator.geocode(address)
        if(location is not None):
            latitude = location.latitude
            longitude = location.longitude
            print('The geocoordinates of {} in {} are {}, {}.'.format(fsa, borough, latitude, longitude))
        else:
            print('No geocoordinates found for {} in {}'.format(fsa, borough))
    

The geocoordinates of M3A in North York are 34.0616262, -117.6495909.
No geocoordinates found for M4A in North York
No geocoordinates found for M5A in Downtown Toronto
No geocoordinates found for M6A in North York
The geocoordinates of M7A in Queen's Park are 34.0616262, -117.6495909.
No geocoordinates found for M9A in Etobicoke
The geocoordinates of M1B in Scarborough are 45.2552743, -76.2897596.
No geocoordinates found for M3B in North York
No geocoordinates found for M4B in East York
No geocoordinates found for M5B in Downtown Toronto
No geocoordinates found for M6B in North York
No geocoordinates found for M9B in Etobicoke
The geocoordinates of M1C in Scarborough are 34.0616262, -117.6495909.
No geocoordinates found for M3C in North York
No geocoordinates found for M4C in East York
No geocoordinates found for M5C in Downtown Toronto
No geocoordinates found for M6C in York
The geocoordinates of M9C in Etobicoke are 43.64410993066079, -79.58890692194828.
No geocoordinates found for M

### Find the FourSquare Venues per Postal Code
The function leveraged from past assignments essentially takes a list of lists of venues for each postal code, flattens it out and creates a dataframe of it. We run it then take all the venues and use pandas to one hot encode them. 

In [12]:
# FourSquare credentials
CLIENT_ID = '0II4MXQK5GVKQA3YIZRXT3D0KWBAKEH2BCCYRWIK4H0DS5XH' # your Foursquare ID
CLIENT_SECRET = 'ZBOCOGUCP2AXOQNAFSPX05IAXWAPBNUBC2FTAGYJV4DDS3AA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [13]:
# This function takes a sequence of names, lats and longs and produces a dataframe of nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(fsa_df['PostalCode'], fsa_df['Latitude'], fsa_df['Longitude'])
toronto_venues.head(10)

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,M4A,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,M5A,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
7,M5A,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,M5A,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
9,M5A,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [15]:
toronto_venues.groupby('Neighborhood')['Venue'].count().to_frame()

,Venue
Neighborhood,
M1B,1
M1C,2
M1E,9
M1G,3
M1H,8
M1J,2
M1K,6
M1L,7
M1M,2


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


### Categorizing Each Postal Code by the Venues
By taking the mean across each postal code you get a measure of what percentage of that postal code is characterized by the venue category. The advantage of this is that the data is already scaled from zero to one. This is the final result we want: each postal code with segmented by its makeup of venue categories.

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [109]:
# Now characterize the postal codes by taking a mean across the one-hot encodings
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [20]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [183]:
# Achieve the final result in a clean dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Accessories Store,Moving Target,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
1,M1C,Construction & Landscaping,Bar,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
2,M1E,Electronics Store,Rental Car Location,Medical Center,Donut Shop,Intersection,Restaurant,Bank,Mexican Restaurant,Breakfast Spot,Mobile Phone Shop
3,M1G,Coffee Shop,Korean BBQ Restaurant,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Accessories Store,Moving Target
4,M1H,Bank,Bakery,Gas Station,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Monument / Landmark,Movie Theater


### Cluster and Visualize Postal Codes
Now to the coup de grace! We'll cluster by FSA and see how the makeup and geospacing can give us insights into their nature. We chose 5 clusters. 

In [184]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [185]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 3, 6, 6, 6, 8, 6, 0, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6, 2, 8,
       2, 6, 6, 6, 6, 2, 6, 7, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 2, 6, 6,
       6, 2, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 2, 6, 6, 6, 6,
       6, 6, 6, 0, 2, 2, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 2,
       2, 6, 0, 6, 8, 7, 4, 8, 8, 6, 6], dtype=int32)

In [186]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = fsa_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='PostalCode')
toronto_merged.dropna(inplace=True) # throw out postal codes for which there is no data
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32') # fix labels

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,6,Intersection,Hockey Arena,Portuguese Restaurant,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Electronics Store,Restaurant,Event Space
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6,Clothing Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,Furniture / Home Store,Coffee Shop,Boutique,Women's Store,Other Great Outdoors,Organic Grocery
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,6,Coffee Shop,Sushi Restaurant,College Cafeteria,Yoga Studio,Theater,Smoothie Shop,Café,Mexican Restaurant,Fried Chicken Joint,Sculpture Garden


In [187]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="robs_ba_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Examine Clusters

This data set needs further development that's out of scope. Cluster 6, i.e. The Blob, kept coming up in analysis despite trying several values of `k`. Also many single node "clusters" were also a feature with several values of `k`. Still Clusters 2, 8, and 0 all exhibited clear features.

| Cluster | Name | Description/Comments |
| --- | --- | --- |
| 6 | Starbucks | This is too large a cluster to categorize properly. There is some factor that isn't being modeled or mischaracterized in the data beyond it's ubiquity across the city like Starbucks. Coffee shops were well represented as a top feature. |
| 2 | Parks & Rec | Parks and other points of interest dominate. Of note was very similar ranks of restaurants among the lower ranked top venues. |
| 8 | Pizza! Pizza! | Dominated by pizza shops |
| 0 | Half-baked | Bakeries seem to dominate but the relationship is weak. |



In [203]:
toronto_merged.groupby('Cluster Labels').count().sort_values(by='PostalCode', ascending=False)['PostalCode'].to_frame()

,PostalCode
Cluster Labels,
6,70
2,13
8,6
0,4
7,2
1,1
3,1
4,1
5,1


In [204]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,6,Intersection,Hockey Arena,Portuguese Restaurant,Coffee Shop,Accessories Store,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
2,M5A,6,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Electronics Store,Restaurant,Event Space
3,M6A,6,Clothing Store,Accessories Store,Vietnamese Restaurant,Miscellaneous Shop,Furniture / Home Store,Coffee Shop,Boutique,Women's Store,Other Great Outdoors,Organic Grocery
4,M7A,6,Coffee Shop,Sushi Restaurant,College Cafeteria,Yoga Studio,Theater,Smoothie Shop,Café,Mexican Restaurant,Fried Chicken Joint,Sculpture Garden
7,M3B,6,Japanese Restaurant,Gym,Athletics & Sports,Café,Dessert Shop,Caribbean Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Museum
8,M4B,6,Pizza Place,Pharmacy,Intersection,Flea Market,Café,Bank,Athletics & Sports,Gastropub,Breakfast Spot,Pet Store
9,M5B,6,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Diner
13,M3C,6,Coffee Shop,Clothing Store,Restaurant,Gym,Art Gallery,Supermarket,Beer Store,Bike Shop,Italian Restaurant,Asian Restaurant
14,M4C,6,Skating Rink,Spa,Park,Athletics & Sports,Beer Store,Curling Ice,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
15,M5C,6,Coffee Shop,Café,Cocktail Bar,Cosmetics Shop,Gastropub,Lingerie Store,Seafood Restaurant,Clothing Store,Moroccan Restaurant,Creperie


In [205]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,2,Park,Food & Drink Shop,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant
16,M6C,2,Field,Park,Hockey Arena,Trail,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
21,M6E,2,Park,Women's Store,Pool,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
35,M4J,2,Park,Coffee Shop,Convenience Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
40,M3K,2,Airport,Park,Accessories Store,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
61,M4N,2,Park,Swim School,Bus Line,Accessories Store,Mobile Phone Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
66,M2P,2,Convenience Store,Park,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Middle Eastern Restaurant
68,M5P,2,Park,Jewelry Store,Sushi Restaurant,Trail,Accessories Store,Mobile Phone Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
83,M4T,2,Tennis Court,Playground,Gym,Park,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
85,M1V,2,Playground,Intersection,Park,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop


In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M1J,8,Spa,Playground,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
50,M9L,8,Pizza Place,Intersection,Gym,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Mobile Phone Shop
70,M9P,8,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Intersection,Discount Store,Middle Eastern Restaurant,Playground,Mobile Phone Shop,Modern European Restaurant
72,M2R,8,Pharmacy,Grocery Store,Discount Store,Coffee Shop,Pizza Place,Outdoor Sculpture,Park,Medical Center,Mediterranean Restaurant,Men's Store
77,M9R,8,Pizza Place,Park,Bus Line,Sandwich Place,Modern European Restaurant,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
93,M8W,8,Pizza Place,Gym,Sandwich Place,Coffee Shop,Pub,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moving Target


In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6B,0,Japanese Restaurant,Sushi Restaurant,Pizza Place,Park,Asian Restaurant,Bakery,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
11,M9B,0,Bakery,Accessories Store,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop,Market
44,M1L,0,Bakery,Soccer Field,Park,Intersection,Ice Cream Shop,Bus Line,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
49,M6L,0,Construction & Landscaping,Bakery,Park,Basketball Court,Modern European Restaurant,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Mobile Phone Shop
